## setup

In [2]:
import pandas as pd
import cv2
import os
import os.path

import time

import requests
import json
import urllib
from bs4 import BeautifulSoup
import re

# show all dataframe
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_columns', None)  

### prototype

### use search page to iterate costumes

In [27]:
# checkout a topic
# by searching on mfa website
# handpick some costume related keywords
keyword = 'dress'
url = f"https://collections.mfa.org/search/Objects/*/{keyword}/images?filter=allClassifications%3ACostumes%3BimageExistence%3Atrue&page=60"
r = requests.get(url)
r

<Response [200]>

In [28]:
soup_key = BeautifulSoup(r.content,'html.parser')

In [29]:
# this is the 12 images div info
k = soup_key.find_all('div',class_="primaryMedia")

In [32]:
# href has the object number for info scraping
# img src has the number of download link
k[11]

<div class="primaryMedia img-wrap"><a href="/objects/535024/dress;jsessionid=ABF7A5FE2B884393F33AB8436009D954?ctx=da4117bb-110e-4447-8f03-83a3cd7eb81f&amp;idx=719" title="Dress"><div class="emuseum-img-wrap width-img-wrap" style="max-height:279px;max-width:279px;"><img class="" src="/internal/media/dispatcher/586267/resize%3Aformat%3Dthumbnail;jsessionid=ABF7A5FE2B884393F33AB8436009D954" title="Dress"/></div><!-- Default (format unspecific) block when media not available --><!-- Access is restricted --><!-- UNKNOWN mimetype --><!-- IMAGE blocks --><!-- AUDIO blocks --><!-- VIDEO blocks --><!-- ZOOMIFY block --><!-- PDF files --><!-- Media info --></a></div>

In [33]:
# find the object number
object_str = k[11].find('a').get('href')
object_str

'/objects/535024/dress;jsessionid=ABF7A5FE2B884393F33AB8436009D954?ctx=da4117bb-110e-4447-8f03-83a3cd7eb81f&idx=719'

In [34]:
# find the download number of this object
download_str = k[0].find('img').get('src')
download_str

'/internal/media/dispatcher/1155015/resize%3Aformat%3Dthumbnail;jsessionid=ABF7A5FE2B884393F33AB8436009D954'

In [35]:
# url of the certain object
url_base = 'https://collections.mfa.org'
object_url = url_base + object_str
print(object_url)


https://collections.mfa.org/objects/535024/dress;jsessionid=ABF7A5FE2B884393F33AB8436009D954?ctx=da4117bb-110e-4447-8f03-83a3cd7eb81f&idx=719


In [36]:
# get the bs for object page
req2 = requests.get(object_url)
soup2 = BeautifulSoup(req2.content,'html.parser')

### if downloadable

In [49]:
# check if the image is downloadable
# which believed to be in public domain
bl = soup2.body.findAll(text='DOWNLOAD')
if bool(bl) is True:
    print('This image is downloadable.')
else:
    print('This image cannot be downloaded.')

This image is downloadable.


### get info from page

In [59]:
# title
soup2.find('div',class_="detailField titleField").text

'Dress'

In [63]:
# year?
soup2.find('div',class_="detailField displayDateField").text.strip()

'about 1735, restyled 1763'

In [82]:
soup2.title

<title>Dress – Results – Advanced Search Objects – Museum of Fine Arts, Boston</title>

In [86]:
soup2.find('div',class_="item-details-inner").get_text()

"Dress\nDress, American (Boston, MA); fabric English\n\nabout 1735, restyled 1763\nObject Place: MA, United StatesMedium/Technique\nSilk; brocaded plain weaveDimensions\n136.52 cm (53 3/4 in.)Credit Line\nGift of Miss Helen L. CampbellAccession Number1990.513a-bNOT ON VIEWCollectionsEurope, Textiles and Fashion ArtsClassificationsCostumesDescriptionRemodeled 1763. Silk: England (Spitalfields), design about 1740. Dark green silk with polychrome brocaded design of flowers, leaves, branches and fruit. Mid-18th century linen bobbin lace stitched inside sleeve flounces and around neck. Open robe à l'anglaise and matching petticoat with deep flounce. Believed to have been worn by Mary Waters of Salem at her marriage to Anthony Sigourney of Boston in 1740. Also worn by their daughter, also named Mary Waters, for her marriage to James Butler in 1763. (Worn with 1990.514a, b). Condition: Petticoat good but fragile. Over dress (robe): good condition; left shoulder seam unstitched, back neck edge

In [214]:
# get the block of item details
tag = soup2.find('div',class_="item-details-inner")
tag

<div class="item-details-inner"><!-- LEFT COLUMN --><div class="col-lg-6"><div class="detailField titleField"><h2>Dress</h2><!--Dress--></div><div class="detailField cultureField">
Dress, American (Boston, MA); fabric English
</div><div class="detailField displayDateField">
about 1735, restyled 1763
</div><div class="detailField objectGeographyField">Object Place: MA, United States<br/><hr class="detailLine"/></div><div class="detailField mediumField"><span class="detailFieldLabel topLabel">Medium/Technique</span><span class="detailFieldValue">
Silk; brocaded plain weave<br/></span></div><div class="detailField dimensionsField != null"><span class="detailFieldLabel topLabel">Dimensions</span><span class="detailFieldValue">
136.52 cm (53 3/4 in.)<br/></span></div><div class="detailField creditlineField != null"><span class="detailFieldLabel topLabel">Credit Line</span><span class="detailFieldValue">
Gift of Miss Helen L. Campbell<br/></span></div><div class="detailField invnolineField">

In [219]:
# locate the block that we need
tag1 = tag.contents[1]
tag1

<div class="col-lg-6"><div class="detailField titleField"><h2>Dress</h2><!--Dress--></div><div class="detailField cultureField">
Dress, American (Boston, MA); fabric English
</div><div class="detailField displayDateField">
about 1735, restyled 1763
</div><div class="detailField objectGeographyField">Object Place: MA, United States<br/><hr class="detailLine"/></div><div class="detailField mediumField"><span class="detailFieldLabel topLabel">Medium/Technique</span><span class="detailFieldValue">
Silk; brocaded plain weave<br/></span></div><div class="detailField dimensionsField != null"><span class="detailFieldLabel topLabel">Dimensions</span><span class="detailFieldValue">
136.52 cm (53 3/4 in.)<br/></span></div><div class="detailField creditlineField != null"><span class="detailFieldLabel topLabel">Credit Line</span><span class="detailFieldValue">
Gift of Miss Helen L. Campbell<br/></span></div><div class="detailField invnolineField"><span class="detailFieldLabel topLabel">Accession Nu

In [220]:
# n of features
len(tag1)

11

In [203]:
object_number = re.search('objects/(.*?)/', object_str).group(1)
object_number

'535024'

In [255]:
# initiate df index
detailField = []
detailField.append('object number')
detailField.append('title')
for i in range(1,11):
    detailField.append(tag1.contents[i]['class'][1][:-5])

df = pd.DataFrame(columns=detailField)
df

Empty DataFrame
Columns: [object number, title, culture, displayDate, objectGeography, medium, dimensions, creditline, invnoline, onview, collectionTerms, classifications]
Index: []

In [256]:
# get object info
detailContent = []

# add object number for record
detailContent.append(object_number)

# the title is special, add it first
detailContent.append(tag1.contents[0].text)

for i in range(1,11):
    detailContent.append(tag1.contents[i].contents[-1].text.strip().replace('\n',''))

print(detailContent)

['535024', 'Dress', 'Dress, American (Boston, MA); fabric English', 'about 1735, restyled 1763', '', 'Silk; brocaded plain weave', '136.52 cm (53 3/4 in.)', 'Gift of Miss Helen L. Campbell', '1990.513a-b', 'NOT ON VIEW', 'Europe, Textiles and Fashion Arts', 'Costumes']


In [257]:
df_addon = pd.DataFrame(detailContent).T
df_addon.columns = detailField
df_addon

object number  title                                       culture  \
0        535024  Dress  Dress, American (Boston, MA); fabric English   

                 displayDate objectGeography                      medium  \
0  about 1735, restyled 1763                  Silk; brocaded plain weave   

               dimensions                      creditline    invnoline  \
0  136.52 cm (53 3/4 in.)  Gift of Miss Helen L. Campbell  1990.513a-b   

        onview                    collectionTerms classifications  
0  NOT ON VIEW  Europe, Textiles and Fashion Arts        Costumes

In [258]:
df = pd.concat([df,df_addon])

In [259]:
df

object number  title                                       culture  \
0        535024  Dress  Dress, American (Boston, MA); fabric English   

                 displayDate objectGeography                      medium  \
0  about 1735, restyled 1763                  Silk; brocaded plain weave   

               dimensions                      creditline    invnoline  \
0  136.52 cm (53 3/4 in.)  Gift of Miss Helen L. Campbell  1990.513a-b   

        onview                    collectionTerms classifications  
0  NOT ON VIEW  Europe, Textiles and Fashion Arts        Costumes

## def func save_image()

In [ ]:
def save_image():
    start = time.time()


In [ ]:
def download_img(df,dir='testing/'):
    start = time.time()
    print(f'{df.shape[0]} images will be downloaded.')

    name = df['objectID']
    url = df['primaryImage']

    name = list(name)
    url = list(url)

    error_id = []
    error_n = 0

    for n,u in zip(name,url):
        filename = dir + str(n) + '.jpg'
        try:
            z = urllib.request.urlopen(u)
            output = open(filename, 'wb')
            output.write(z.read())
            output.close()
        except:
            error_id.append(n)
            print(f'error id {n}')
            error_n += 1
            print(f'{error_n} errors in total')
            continue


    end = time.time()
    print(f'{end - start} secs slipped...')

In [6]:
x = soup.find_all('img')

In [7]:
t = list(x)

In [9]:
len(t)

13

In [8]:
# on one page there are 12 thumbnail images
# of the costumes we want
# the first one is unrelated, we should get rid of it
t[1:]

[<img class="" src="/internal/media/dispatcher/1544048/resize%3Aformat%3Dthumbnail;jsessionid=1A1A5B7352C444E0A2FB8651FF5CD4B7" title="Pinned dress and silver pin"/>,
 <img class="" src="/internal/media/dispatcher/1577155/resize%3Aformat%3Dthumbnail;jsessionid=1A1A5B7352C444E0A2FB8651FF5CD4B7" title="Look 7: Dress, pants, and ankle boots"/>,
 <img class="" src="/internal/media/dispatcher/1544173/resize%3Aformat%3Dthumbnail;jsessionid=1A1A5B7352C444E0A2FB8651FF5CD4B7" title="Woman's ensemble comprising dress, tights, shoes, and spats"/>,
 <img class="" src="/internal/media/dispatcher/1577158/resize%3Aformat%3Dthumbnail;jsessionid=1A1A5B7352C444E0A2FB8651FF5CD4B7" title="Woman's ensemble comprising dress, hood, and boots"/>,
 <img class="" src="/internal/media/dispatcher/1544063/resize%3Aformat%3Dthumbnail;jsessionid=1A1A5B7352C444E0A2FB8651FF5CD4B7" title="Sculpted dress and shoulder harness"/>,
 <img class="" src="/internal/media/dispatcher/1596815/resize%3Aformat%3Dthumbnail;jsessioni

In [10]:
# we need to locate the numbers after despather
# it's the object name we need for downloading
for link in t[1:]:
    found = re.search('dispatcher/(.*?)/',link.get('src')).group(1)
    print(found)
    # print(link.get('src'))

1544048
1577155
1544173
1577158
1544063
1596815
1634455
1634458
1588174
1590035
1590036
1590038


In [66]:
# the download ref number is different than object number
object_num = '815857'
download_page = f'https://collections.mfa.org/download/{object_num}'
download_page

'https://collections.mfa.org/download/815857'

In [68]:
# get the full size download link from the download page
url = download_page
req = requests.get(url)
soup = BeautifulSoup(req.content,'html.parser')
soup.find_all('a','target'="_blank")


# <a target="_blank" href="https://collections.mfa.org/internal/media/dispatcher/757433/resize%3Aformat%3Dfull?download"><button class="emuseum-button " type="button">
# Download
# </button></a>

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (2056816795.py, line 5)

In [64]:

download_link = f'https://collections.mfa.org/internal/media/dispatcher/{object_num}/resize%3Aformat%3Dfull?download'

print(download_link)

https://collections.mfa.org/internal/media/dispatcher/815857/resize%3Aformat%3Dfull?download


In [65]:
z = urllib.request.urlopen(download_link)
output = open('test.jpg', 'wb')
output.write(z.read())
output.close()

## get the dataframe for selected topics

### prototoype to scrape a certain topic

In [5]:
# continue above

# get records column names
# for initial the full dataframe
records_list = r.json()['records']
records_df = pd.DataFrame.from_dict(records_list)
records_df_columns = records_df.columns

# initiate a dataframe for filling in scrapped info
df = pd.DataFrame(columns=records_df_columns)

In [6]:
df # check if initiated

Empty DataFrame
Columns: [systemNumber, accessionNumber, objectType, _currentLocation, _primaryTitle, _primaryMaker, _primaryImageId, _primaryDate, _primaryPlace, _warningTypes, _images]
Index: []

In [7]:
keyword = 'dress'

In [8]:
urlpath = f'https://api.vam.ac.uk/v2/objects/search?q_object_name={keyword}&page_size=100&images=true&images_exist=true'
rr = requests.get(urlpath)

info_dict = rr.json()['info'] 
pages_n = info_dict['pages']
record_n = info_dict['record_count']
print(f'{pages_n} pages for keyword {keyword}')
print(f'{record_n} records for keyword {keyword}')

74 pages for keyword dress
7321 records for keyword dress


In [9]:
# start timer
start = time.time()

for p in range(1,pages_n+1):
    urlpath = f'https://api.vam.ac.uk/v2/objects/search?q_object_name={keyword}&page_size=100&images=true&images_exist=true&page={p}'
    rr = requests.get(urlpath)
    thisdf = pd.DataFrame.from_dict(rr.json()['records'])
    df = pd.concat([df,thisdf])

# timer stops
end = time.time()
print(f'{end-start} seconds passed..')

47.67231202125549 seconds passed..


In [10]:
df.head(1)

systemNumber accessionNumber objectType  \
0       O63107      T.237-2001      Dress   

                                                                                                                                                  _currentLocation  \
0  {'id': 'THES50141', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}   

  _primaryTitle                                                _primaryMaker  \
0         Dress  {'name': 'Tam, Vivienne', 'association': 'Maker, Designer'}   

  _primaryImageId _primaryDate _primaryPlace _warningTypes  \
0      2020MU3350         1999           USA            []   

                                                                                                                                                                                                                                             _images  
0  {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2020MU3350/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2020MU3350/', '_iiif_presentation_url': None, 'imageResolution': 'low'}

### define the function

In [39]:
def scrape_dataframe(keyword='robe'):
    # start timer
    start = time.time()    

    urlpath = f'https://api.vam.ac.uk/v2/objects/search?q_object_name={keyword}&page_size=100&images=true&images_exist=true'
    rr = requests.get(urlpath)

    # initiate an empty dataframe to carry the scrapped info
    # get records column names
    # for initial the full dataframe
    records_list = rr.json()['records']
    records_df = pd.DataFrame.from_dict(records_list)
    records_df_columns = records_df.columns
    df = pd.DataFrame(columns=records_df_columns)

    # get how many pages
    # and total number of records of this keyword
    info_dict = rr.json()['info'] 
    pages_n = info_dict['pages'] # for iterate
    record_n = info_dict['record_count']
    # print(f'{pages_n} pages for keyword {keyword}')
    print(f'we found {record_n} records for keyword {keyword}')

    # iterate the pages
    # since VA API could only search page by page
    for p in range(1,pages_n+1):
        urlpath = f'https://api.vam.ac.uk/v2/objects/search?q_object_name={keyword}&page_size=100&images=true&images_exist=true&page={p}'
        rr = requests.get(urlpath)
        thisdf = pd.DataFrame.from_dict(rr.json()['records'])
        df = pd.concat([df,thisdf])

    # print(f'we got {df.shape[0]} scarf.')
    
    # timer stops
    end = time.time()
    print(f'{end-start} seconds for scraping {keyword}..')

    return df

In [40]:
# test the function
df_test = scrape_dataframe('scarf')
df_test.head(1)

we found 782 records for keyword scarf
5.681551933288574 seconds for scraping scarf..


systemNumber accessionNumber objectType  \
0      O102827      T.127-1997      Scarf   

                                                                                                                                                  _currentLocation  \
0  {'id': 'THES50141', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}   

    _primaryTitle                                           _primaryMaker  \
0  Scarf on Scarf  {'name': 'Earley, Rebecca', 'association': 'designer'}   

  _primaryImageId _primaryDate _primaryPlace _warningTypes  \
0      2006AP6466         1996       England            []   

                                                                                                                                                                                                                                             _images  
0  {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2006AP6466/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2006AP6466/', '_iiif_presentation_url': None, 'imageResolution': 'low'}

In [41]:
df_test = df_test.head(3)

### scrape

In [15]:
# initiate an empty datafram
url = "https://api.vam.ac.uk/v2/objects/search?q_object_name=robe&page_size=100&images=true&images_exist=true"
r = requests.get(url)

# get records column names
# for initial the full dataframe
records_list = r.json()['records']
records_df = pd.DataFrame.from_dict(records_list)
records_df_columns = records_df.columns

# initiate a dataframe for filling in scrapped info
df = pd.DataFrame(columns=records_df_columns)

In [16]:
#check
df

Empty DataFrame
Columns: [systemNumber, accessionNumber, objectType, _currentLocation, _primaryTitle, _primaryMaker, _primaryImageId, _primaryDate, _primaryPlace, _warningTypes, _images]
Index: []

In [17]:
# hand picked some keywords
# checked the keywords on VA website

keywords = ['costume', 'robe', 'dress', 'shirt','ensemble','jacket','coat','suit','trousers']

In [18]:
for k in keywords:
    df_toadd = scrape_dataframe(k)
    df = pd.concat([df,df_toadd])

we found 16896 records for keyword costume
65.27326703071594 seconds for scraping costume..
we found 5582 records for keyword robe
35.6450834274292 seconds for scraping robe..
we found 7321 records for keyword dress
48.430097341537476 seconds for scraping dress..
we found 917 records for keyword shirt
6.8267552852630615 seconds for scraping shirt..
we found 399 records for keyword ensemble
3.2287628650665283 seconds for scraping ensemble..
we found 835 records for keyword jacket
6.3819639682769775 seconds for scraping jacket..
we found 763 records for keyword coat
5.62924599647522 seconds for scraping coat..
we found 979 records for keyword suit
7.155668497085571 seconds for scraping suit..
we found 360 records for keyword trousers
3.063270330429077 seconds for scraping trousers..


In [19]:
df.shape

(27156, 11)

In [20]:
df.drop_duplicates('_primaryImageId', inplace=True)

In [21]:
df.to_csv(r'scrapped_data/VA_uncleaned.csv', index = False)

In [2]:
df = pd.read_csv('scrapped_data/VA_uncleaned.csv') 

In [3]:
df.shape

(26595, 11)

## filter more before download

### hand pick certian object types

after download test, we found many unrealted images.

we could use 'object type' to filter.<br>
use the VA website search to determine if the object type needs to be removed.

In [4]:
df['objectType'] = df['objectType'].map(lambda x: str(x).lower())

In [5]:
df['objectType'].nunique()

584

In [6]:
df['objectType'].unique()

array(['costume', 'theatre costume', 'film costume', 'costume design',
       'ballet costume', 'opera costume', 'dance costume',
       'puppet costume', 'stage costume', 'fancy dress costume',
       'circus costume', 'costume plate', 'clown costume',
       'rock and pop costume', 'jacket', 'print', 'drawing',
       'fashion plate', 'nan', "doll's outfit", 'ensemble', 'bodice',
       'theatre design', 'figurine', 'oil painting', 'design',
       'aquatint etching', 'statuette', 'photograph', 'poster', 'figure',
       'relief', 'screenprint', 'fashion design', 'costume sketch',
       'costume doll', 'masks (costume)', 'cabaret costume',
       'costume accessory', 'marionette costume', 'walking costume',
       'bathing costume', 'national costume', 'golf costume',
       'costume designs', 'costume dolls', 'usher costume',
       'performance costume', 'costume print', 'tv costume',
       "man's costume", "acrobat's costume", 'theatrical costume',
       'costume doll ', 'robe'

In [15]:
df['objectType'].value_counts().head(60)

costume design                7379
fashion design                5388
dress fabric                  3774
theatre costume               1723
dress                         1141
evening dress                  638
jacket                         552
shirt                          437
coat                           379
t-shirt                        338
print                          334
ensemble                       332
trousers                       276
suit                           233
costume                        207
robe                           188
skirt suit                     175
drawing                        159
photograph                     140
wedding dress                  134
dress panel                    130
dance costume                  115
fashion plate                   94
trouser suit                    68
dress ornament                  65
day dress                       62
dress trimming                  61
book jacket                     61
costume doll        

In [16]:
sel = ['costume design', 'fashion design','dress fabric','photograph',
        'print','drawing','dress panel','costume doll',
        'dress trimming','dress-making sample','book jacket','painting',
        'fancy dress costume design','watercolour','etching','costume sketch',
        'oil painting','costume plate','theatre costume']

In [17]:
for s in sel:
    sel_index = df[df['objectType'] == s].index
    df.drop(sel_index, inplace=True)

In [18]:
df.shape

(7096, 11)

In [24]:
# easy check an object
sysnumber = 'O1682165'
df[df['systemNumber']==sysnumber]

systemNumber  accessionNumber objectType  \
3     O1682165  S.8:1 to 5-2022    costume   

                                                                                                                                                                                  _currentLocation  \
3  {'id': 'THES49929', 'displayName': 'Theatre & Performance, Room 104', 'type': 'Display', 'site': 'VA', 'onDisplay': True, 'detail': {'free': '', 'case': 'PL1', 'shelf': 'EXP', 'box': 'FIG1'}}   

  _primaryTitle                                            _primaryMaker  \
3       costume  {'name': 'Slade, Gabriella', 'association': 'designer'}   

  _primaryImageId _primaryDate _primaryPlace _warningTypes  \
3      2022NG6373         2020      New York            []   

                                                                                                                                                                                                                                                                                                     _images  
3  {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2022NG6373/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2022NG6373/', '_iiif_presentation_url': 'https://iiif.vam.ac.uk/collections/O1682165/manifest.json', 'imageResolution': 'high'}

## image download

### extract base url

In [25]:
df['image_value'] = df['_images'].map(lambda x: eval(x))

In [26]:
df['image_value'] = df['image_value'].map(lambda x: x.values())

In [27]:
df['image_base'] = df['image_value'].map(lambda x: (list(x)[1]))

In [28]:
df.head(1)

systemNumber accessionNumber objectType  \
0     O1314778     S.1658-2015    costume   

                                                                                                                                                  _currentLocation  \
0  {'id': 'THES49318', 'displayName': 'In store', 'type': 'storage', 'site': 'BH', 'onDisplay': False, 'detail': {'free': '', 'case': '', 'shelf': '', 'box': ''}}   

  _primaryTitle                                           _primaryMaker  \
0       Costume  {'name': 'Strassner, Joe', 'association': 'designers'}   

  _primaryImageId _primaryDate  _primaryPlace _warningTypes  \
0      2015HV9075         1936  Great Britain            []   

                                                                                                                                                                                                                                             _images  \
0  {'_primary_thumbnail': 'https://framemark.vam.ac.uk/collections/2015HV9075/full/!100,100/0/default.jpg', '_iiif_image_base_url': 'https://framemark.vam.ac.uk/collections/2015HV9075/', '_iiif_presentation_url': None, 'imageResolution': 'low'}   

                                                                                                                                        image_value  \
0  (https://framemark.vam.ac.uk/collections/2015HV9075/full/!100,100/0/default.jpg, https://framemark.vam.ac.uk/collections/2015HV9075/, None, low)   

                                            image_base  
0  https://framemark.vam.ac.uk/collections/2015HV9075/

In [44]:
df_test['image_value'] = df_test['_images'].map(lambda x: x.values())
df_test['image_base'] = df_test['image_value'].map(lambda x: (list(x)[1]))

### prototype

In [ ]:
test_sample = df.sample(1,random_state = 2333)
test_sample.columns

Index(['systemNumber', 'accessionNumber', 'objectType', '_currentLocation',
       '_primaryTitle', '_primaryMaker', '_primaryImageId', '_primaryDate',
       '_primaryPlace', '_warningTypes', '_images'],
      dtype='object')

In [ ]:
image_url_dict = test_sample['_images'].to_dict()

In [ ]:
k = list(image_url_dict.keys())
k=k[0]

In [ ]:
image_base_url = image_url_dict[k]['_iiif_image_base_url']

In [ ]:
image_base_url

'https://framemark.vam.ac.uk/collections/2011EP4395/'

In [ ]:
image_download_url = image_base_url + 'full/full/0/default.jpg'
image_download_url

'https://framemark.vam.ac.uk/collections/2011EP4395/full/full/0/default.jpg'

In [ ]:
# get the name of the file
# create the filepath to save the image
dir = 'clean image/VA/'
systemNumber = test_sample['systemNumber'].to_string(index=False)
filepath = dir + systemNumber + '.jpg'

print(filepath)

clean image/VA/O1174504.jpg


In [ ]:
# save the image
z = urllib.request.urlopen(image_download_url)
output = open(filepath, 'wb')
output.write(z.read())
output.close()

In [ ]:
df_test['image_base']

0    https://framemark.vam.ac.uk/collections/2006AP6466/
1    https://framemark.vam.ac.uk/collections/2019LM0853/
2    https://framemark.vam.ac.uk/collections/2019LM1011/
Name: image_base, dtype: object

### define the function

In [31]:
def download_image(dir = 'clean image/VA/', df=df):
    start = time.time()
    print(f'{df.shape[0]} images will be downloaded.')    

    name = df['systemNumber']
    url = df['image_base']
    name = list(name)
    url = list(url)

    error_id = []
    error_n = 0

    # check if the folder existed
    # if not, create the folder
    b = os.path.exists(dir)
    if b == False:
        os.mkdir(dir) 

    for n,u in zip(name,url):
        filename = dir + n + '.jpg'
        uu = u + 'full/full/0/default.jpg'
        try:
            z = urllib.request.urlopen(uu)
            output = open(filename, 'wb')
            output.write(z.read())
            output.close()
        except:
            error_id.append(n)
            print(f'error id {n}')
            error_n += 1
            print(f'{error_n} errors in total')
            continue

    end = time.time()
    print(f'{end - start} secs slipped...')

In [45]:
# test
download_image('vatest/', df_test)

3 images will be downloaded.
3.106887102127075 secs slipped...


### scrape

In [33]:
df.shape

(7096, 13)

In [32]:
download_image('clean image/VA_sel/', df)

7096 images will be downloaded.
9284.307857513428 secs slipped...


In [34]:
df.to_csv(r'scrapped_data/VA_all_sel.csv', index = False)

In [35]:
df = pd.read_csv('scrapped_data/VA_all_sel.csv') 